In [ ]:
import sys
sys.path.append("..")

In [ ]:
import Graph
import Skeletonize
import MNIST
from importlib import reload
from scipy.sparse import csr_matrix
import numpy as np
import os
import time
from scipy.spatial.distance import cdist

reload(Graph)
reload(Skeletonize)
reload(MNIST)

In [ ]:
def cur_time():
    t = time.localtime()
    return f"{t.tm_hour}:{t.tm_min}:{t.tm_sec}"

In [ ]:
size = 60000
forget_percentage=[.9, .99, .999]
SAVE_PATH = r"F:\Git\Bachelor2.0\Data\Experiments\VaryingLabels"
Ks = [x for x in range(2, 41, 2)]

In [ ]:
train_x, train_Y, embedding = Graph.load_embedding(size)
test_x, test_Y = MNIST.load_pymde_testset()
validation_x, validation_Y = test_x, test_Y
lp_test_result = []

dist = cdist(validation_x, train_x, metric="euclidean")

In [ ]:

for k in Ks:
    g = Graph.construct_knn_graph_3(embedding, k=k)
    for fp in forget_percentage:
        oht = Graph.one_hot_encode_labels(train_Y)
        oht, forg_indices = Graph.forget_oht_labels(oht, forget_percentage=fp)

        forg_labels = np.copy(train_Y)
        forg_labels[forg_indices] = -1

        sparse_oht = csr_matrix(oht)
        new_lbls = Graph.propagate_labels(g, sparse_oht, max_itter=30)

        true_lbls = train_Y[forg_indices]
        pred_lbls = new_lbls[forg_indices]

        reconstruction_acc = Graph.test_accuracy(true_lbls=true_lbls, pred_labels=pred_lbls)

        test_acc = Graph.test_nums_accuracy(train_x, validation_x, new_lbls, validation_Y, k=k, dist=dist)
        print(f"\n####################\n{k=} -- {cur_time()}\n{fp=}\n{reconstruction_acc=}\n{test_acc=}\n####################\n")

        lp_test_result.append([k, fp, reconstruction_acc, test_acc])

In [ ]:
num_of_tries = len([x for x in os.listdir(SAVE_PATH) if "test_lp_" in x])
np.savetxt(os.path.join(SAVE_PATH, f"test_lp_{num_of_tries}.csv"), lp_test_result, delimiter=",", fmt="%f", header="Num neighbors, forgot_percentage, reconstruction, test")

In [ ]:
# train_x, train_Y, embedding = Graph.load_embedding(size)
# test_x, test_Y = MNIST.load_pymde_testset()
# validation_x, validation_Y = test_x[:5000], test_Y[:5000]
srg_test_result = []

In [ ]:
for k in Ks:
    g = Graph.construct_knn_graph_3(embedding, k=k)
    pg = Skeletonize.from_matrix_embedding_to_pygel(g, embedding)
    spg, smap = Skeletonize.load_skeleton_and_smap(k)
    reduction = len(spg.nodes())
    for p in forget_percentage:
        oht = Graph.one_hot_encode_labels(train_Y)
        oht, forg_indices = Graph.forget_oht_labels(oht, forget_percentage=p)

        forg_labels = np.copy(train_Y)
        forg_labels[forg_indices] = -1

        sparse_oht = csr_matrix(oht)

        smg, slbls = Skeletonize.from_pygel_to_matrix(spg, forg_labels, smap)
        prop_labels = Graph.propagate_labels(smg, slbls)
        relbls = Skeletonize.remap_labels(prop_labels, smap)

        true_lbls = train_Y[forg_indices]
        pred_lbls = relbls[forg_indices]

        reconstruction_acc = Graph.test_accuracy(true_lbls, pred_lbls)

        test_acc = Graph.test_nums_accuracy(train_x, validation_x, relbls, validation_Y, k=k, dist=dist)

        srg_test_result.append([k, p, reconstruction_acc, test_acc, reduction])
        print(f"\n####################\n{k=} -- {cur_time()}\n{p=}\n{reconstruction_acc=}\n{test_acc=}\n{reduction=}\n####################\n")

In [ ]:
print(true_lbls.shape, pred_lbls.shape)
print(forg_indices.shape)
print(relbls[forg_indices].shape)

In [ ]:
num_of_tries = len([x for x in os.listdir(SAVE_PATH) if "test_srg_" in x])
np.savetxt(os.path.join(SAVE_PATH, f"test_srg_{num_of_tries}.csv"), srg_test_result, delimiter=",", fmt="%f", header="Num neighbors, forgot_percentage, reconstruction, test, reduction")